# Imports

In [1]:
# Only uncomment if you haven't had these installed before
# !pip install numpy pandas

In [2]:
import numpy as np
import pandas as pd

# Import Data

You should have 4 spreadsheets in the `Data` folder with the following columns:

**Data/flowsheets.csv**
* HSH_ADMSN_ID
* FLO_MEAS_NAME
* NORMALIZED_NAME
* NFLO_MEAS_ID
* GROUPED_FLO_ID
* MEAS_VALUE
* UNITS
* RECODED_TIME


**Data/Labs.csv**
* HSH_ADMSN_ID
* NORMALIZED_LAB_NAME
* COMPONENT_ID
* COMPONENT_NAME
* ORD_VALUE
* ORD_NUM_VALUE_CORRECTED
* RESULT_TIME


**Data/flowsheetspressures.csv**
* HSH_ADMSN_ID
* FLO_MEAS_NAME
* NORMALIZED_NAME
* NFLO_MEAS_ID
* GROUPED_FLO_ID
* MEAS_VALUE
* UNITS
* RECODED_TIME
* SYSTOLIC
* DIASTOLIC
* MAP


**Data/main.csv**
* HSH_ADMSN_ID
* ADMSN_MINUTES
* ICU_MINS
* DISCHARGE_MINUTES
* IN_ICU_TIME
* FIRST_LOCATION
* PROC_NAME
* EXAM_BEGIN_MINUTES
* VERAPAMIL_TAKEN_TIME
* CPT_COIL
* INPT_DEATH_YN
* BMI
* AGE_LT90
* SEX
* ETHNICITY
* RACE
* ADMITTING_SERVICE
* MEANS_OF_ARRIVAL

In [3]:
flowsheets = pd.read_csv('Data/flowsheets.csv')
labs = pd.read_csv('Data/Labs.csv')
flowsheets_pressures = pd.read_csv('Data/flowsheetspressures.csv')
pts = pd.read_csv('Data/main.csv')

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# standardize names of the tests in the flowsheets file
mapping_dict = {
        'O2 FLOW':'Supplemental O2', 
        'FIO2':'Supplemental FIO2',
        'ETCO2': 'Supplemental ETCO2',
        'AIRWAY VIEW GRADE ': 'Mechanical Ventilation',
        'AIRWAY DIFFICULTY': 'Mechanical Ventilation',
        'ETT NUMBER OF ATTEMPTS': 'Mechanical Ventilation',
        'ETT EQUIPMENT': 'Mechanical Ventilation',
        'O2 FACE MASK': 'Supplemental O2',
        'TUBE OUTPUT': 'TUBE FEEDING',
        'MAINTENANCE IV VOLUME': 'IV Fluids',
        'O2 NASAL': 'Supplemental O2',
        'BLOOD ADMINISTRATION VOLUME': 'Blood Admin',
        'BLOOD OUTPUT': 'Blood Loss',
        'AIRWAY COMMENTS': 'Mechanical Ventilation',
        'GASTRIC TUBE': 'TUBE FEEDING'
    }

flowsheets.NORMALIZED_NAME.replace(
    to_replace=mapping_dict, 
inplace=True)


# Filter step 1: 
* This next set of code will solated the needed columns and establish time cutpoints for which the algorithm considers data until.

In [5]:
print("Filtering data...")

Filtering data...


In [6]:
pts_filtered = pts[['HSH_ADMSN_ID', 'DISCHARGE_MINUTES', 'VERAPAMIL_TAKEN_TIME', 
                    'IN_ICU_TIME', 'ICU_MINS', 'BMI', 'AGE_LT90', 
                    'SEX', 'ETHNICITY', 'RACE']]
# Relabel patients as taken vs not taken VERAPAMIL
pts_filtered.loc[~pts_filtered['VERAPAMIL_TAKEN_TIME'].isna(), 'VERAPAMIL_TAKEN'] = True
pts_filtered.loc[pts_filtered['VERAPAMIL_TAKEN_TIME'].isna(), 'VERAPAMIL_TAKEN'] = False

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [7]:
pts_filtered['VERAPAMIL_TAKEN_TIME'] = pts_filtered['VERAPAMIL_TAKEN_TIME'].fillna(pts_filtered['ICU_MINS'] + pts_filtered['IN_ICU_TIME'])

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
pts_filtered['VERAPAMIL_TAKEN_TIME'] = pd.to_numeric(pts_filtered['VERAPAMIL_TAKEN_TIME'])

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
for t in [240, 1440, 2880, 4320, 7200, 10080, 14400, 20160]:
    pts_filtered[f"{t}_cutpoint"] = pts_filtered['VERAPAMIL_TAKEN_TIME'] - t

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Merge data

In [10]:
print("Merging data...")

Merging data...


In [11]:
flowsheets = flowsheets[['HSH_ADMSN_ID', 'NORMALIZED_NAME', 'MEAS_VALUE', 'RECODED_TIME']]

In [12]:
flowsheets_pressures = flowsheets_pressures[['HSH_ADMSN_ID', 'NORMALIZED_NAME', 'SYSTOLIC', 'DIASTOLIC', 'MAP', 'RECODED_TIME']]

In [13]:
countlabs = labs.groupby('HSH_ADMSN_ID').agg('count')

In [14]:
# filter out labs with no data
lab_filtered = countlabs[countlabs['NORMALIZED_LAB_NAME'] > 0].reset_index().rename(
    columns={'NORMALIZED_LAB_NAME': 'lab_count'})[['HSH_ADMSN_ID', 'lab_count']]

In [15]:
labs_clean = labs.merge(lab_filtered, on='HSH_ADMSN_ID', how='right')[['HSH_ADMSN_ID','NORMALIZED_LAB_NAME', 'ORD_NUM_VALUE_CORRECTED', 'RESULT_TIME']]

In [16]:
cutpoints = pts_filtered[['HSH_ADMSN_ID', '240_cutpoint', '1440_cutpoint', '2880_cutpoint', '4320_cutpoint', 
                          '7200_cutpoint', '10080_cutpoint', '14400_cutpoint', '20160_cutpoint']]

In [17]:
flowsheets_w_cutpoints = flowsheets.merge(cutpoints, on='HSH_ADMSN_ID', how='left')

In [18]:
flowsheets_pressures_w_cutpoints = flowsheets_pressures.merge(cutpoints, on='HSH_ADMSN_ID', how='left')

In [19]:
labs_clean_w_cutpoints = labs_clean.merge(cutpoints, on='HSH_ADMSN_ID', how='left')

In [20]:
# Helper
def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f
    return decorator

def q_at(y):
    @rename(f'q{y:0.2f}')
    def q(x):
        return x.quantile(y)
    return q

In [21]:
def output_full_sheet(T):    

    print("Processing Data for t =", T, "minutes")
    l = labs_clean_w_cutpoints.copy()

    l_time = l[l['RESULT_TIME'] < l[f'{T}_cutpoint']] #only get labs before cutpoint time

    f = {'ORD_NUM_VALUE_CORRECTED': ['min', 'max', 'mean', 'count']} # setup for aggregation of labs. didn't do 21 feat vector...too long

    lab_quantiles = l_time.groupby(['HSH_ADMSN_ID', 'NORMALIZED_LAB_NAME']).agg(f) # gets min, max, mean, and count for each lab

    lab_quantiles.columns = lab_quantiles.columns.to_flat_index() # flatten to prep for pivot

    lab_quantiles_flat = lab_quantiles.reset_index() # reset index to prep for pivot

    wide_labs = lab_quantiles_flat.pivot(index='HSH_ADMSN_ID', columns='NORMALIZED_LAB_NAME') 

    wide_labs.columns = wide_labs.columns.to_flat_index()

    print("process ICP data")

    icp = flowsheets_w_cutpoints.copy()

    icp_time = icp[icp['RECODED_TIME'] < icp[f'{T}_cutpoint']] # only get icp before cutpoint time

    icp = icp[icp['NORMALIZED_NAME'] == 'ICP'] # only care about ICP data in this 
 
    icp['ICP_MEAS_VALUE'] = pd.to_numeric(icp['MEAS_VALUE']) # some non numbers, just coerce to 0

    f = {'ICP_MEAS_VALUE': [q_at(x) for x in np.linspace(0.05,1,20)]+['count']} # grouping stuff to get the 20 percentiles

    icp_quantiles = icp.groupby(['HSH_ADMSN_ID']).agg(f) 

    icp_quantiles.columns = icp_quantiles.columns.to_flat_index() 

    icp_quantiles_no_lt100 = icp_quantiles 

    print("process BP data")

    bp = flowsheets_pressures_w_cutpoints.copy() 

    bp = bp[bp['RECODED_TIME'] < bp[f'{T}_cutpoint']] # only stuff after the cutpoint

    f = {'MAP': [q_at(x) for x in np.linspace(0.05,1,20)]+['count']} # same 20 percentile thing thing

    bp_quantiles = bp.groupby(['HSH_ADMSN_ID','NORMALIZED_NAME']).agg(f)

    bp_quantiles.columns = bp_quantiles.columns.to_flat_index()

    bp_quantiles_flat = bp_quantiles.reset_index()

    wide_bp = bp_quantiles_flat.pivot(index='HSH_ADMSN_ID', columns='NORMALIZED_NAME')

    wide_bp.columns = wide_bp.columns.to_flat_index()

    print("get the counts for the variables etc.")

    icp_analysis_one = icp_time.groupby(['HSH_ADMSN_ID', 'NORMALIZED_NAME']).count()[['MEAS_VALUE']]

    icp_analysis_one_raw = icp_analysis_one.reset_index().pivot(index='HSH_ADMSN_ID', columns='NORMALIZED_NAME')

    icp_analysis_one_raw.columns = icp_analysis_one_raw.columns.to_flat_index()

    icp_analysis_bool = icp_analysis_one_raw.notnull()

    icp_analysis_count = icp_analysis_one_raw.fillna(0)

    print("take care of demographics and finally code if someone got varap or not")

    demos = pts_filtered[['HSH_ADMSN_ID', 'VERAPAMIL_TAKEN_TIME', 'VERAPAMIL_TAKEN', 'BMI', 'AGE_LT90', 'SEX', 'ETHNICITY', 'RACE']]

    # demos.loc[demos['VERAPAMIL_TAKEN_TIME'] == 99999999.0, 'VERAPAMIL_TAKEN'] = False

    # demos.loc[demos['VERAPAMIL_TAKEN_TIME'] < 99999999, 'VERAPAMIL_TAKEN'] = True

    demos_labeled = demos[['HSH_ADMSN_ID', 'VERAPAMIL_TAKEN', 'BMI', 'AGE_LT90', 'SEX', 'ETHNICITY', 'RACE']]
    
    """
    variables:
    1. mixed vars binary = icp_analysis_bool
    2. mixed vars counts = icp_analysis_count
    3. ICP actual values = icp_quantiles_no_lt100
    4. BP = wide_bp
    5. lab = wide_labs (use this as the main index probs)
    6. demographics = demos_labeled
    """
    print("merging..")
    final_df = wide_labs.reset_index().merge(demos_labeled, how='left', on='HSH_ADMSN_ID')\
        .merge(wide_bp, how='left', on='HSH_ADMSN_ID')\
        .merge(icp_quantiles_no_lt100, how='left', on='HSH_ADMSN_ID')\
        .merge(icp_analysis_count, how='left', on='HSH_ADMSN_ID')\
        .merge(icp_analysis_bool, how='left', on='HSH_ADMSN_ID')
    print(f"saving to Cleaned/{T}.csv")
    final_df.to_csv(f'Cleaned/{T}.csv')
    print("==Value counts==")
    print(final_df['VERAPAMIL_TAKEN'].value_counts())
    print("====")
    return final_df

In [22]:
df_240 = output_full_sheet(240)

Processing Data for t = 240 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/240.csv
==Value counts==
False    1817
True      125
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [23]:
df_1440 = output_full_sheet(1440)

Processing Data for t = 1440 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/1440.csv
==Value counts==
False    1624
True      118
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [24]:
df_2880 = output_full_sheet(2880)

Processing Data for t = 2880 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/2880.csv
==Value counts==
False    1282
True      111
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [25]:
df_4320 = output_full_sheet(4320)

Processing Data for t = 4320 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/4320.csv
==Value counts==
False    1059
True      103
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [26]:
df_7200 = output_full_sheet(7200)

Processing Data for t = 7200 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/7200.csv
==Value counts==
False    828
True      85
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [27]:
df_10080 = output_full_sheet(10080)

Processing Data for t = 10080 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/10080.csv
==Value counts==
False    713
True      63
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [28]:
df_14400 = output_full_sheet(14400)

Processing Data for t = 14400 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/14400.csv
==Value counts==
False    561
True      34
Name: VERAPAMIL_TAKEN, dtype: int64
====


In [29]:
df_20160 = output_full_sheet(20160)

Processing Data for t = 20160 minutes
process ICP data
process BP data
get the counts for the variables etc.
take care of demographics and finally code if someone got varap or not
merging..
saving to Cleaned/20160.csv
==Value counts==
False    425
True      12
Name: VERAPAMIL_TAKEN, dtype: int64
====


## Adjust csvs

In [30]:

for t in [240, 1440, 2880, 4320, 7200, 10080, 14400, 20160]:
    predictors = pd.read_csv('annotated_predictors.csv')
    inc_columns = predictors[predictors['physiologic'] == 1.0]['HSH_ADMSN_ID'].to_numpy()
    min_columns = filter(lambda x: x.find('count') == -1 and x.find('_x') == -1, inc_columns)
    csv_orig = pd.read_csv(f'Cleaned/{t}.csv')
    csv_orig[csv_orig.columns & (['HSH_ADMSN_ID', 'VERAPAMIL_TAKEN'] + list(min_columns))].to_csv(f'Cleaned/{t}_nocounts.csv')

/Users/abhi/.pyenv/versions/3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  
